In [4]:
!pip install opencv-python

In [5]:
import cv2

In [6]:
!pip install transformers

In [7]:
from transformers import YolosFeatureExtractor

In [8]:
from transformers import YolosForObjectDetection

In [9]:
import torch
from torchvision.ops import nms
from PIL import Image

In [10]:
feature_extractor = YolosFeatureExtractor.from_pretrained("hustvl/yolos-tiny")
model = YolosForObjectDetection.from_pretrained("hustvl/yolos-tiny")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/yolos/feature_extraction_yolos.py:38: FutureWarning: The class YolosFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use YolosImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/26.0M [00:00<?, ?B/s]

In [11]:
video_path = "video.mp4"
cap = cv2.VideoCapture(video_path)

In [12]:
output_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

In [13]:
out = cv2.VideoWriter(output_path, fourcc, 12.0, (int(cap.get(3)), int(cap.get(4))))

In [14]:
target_width = 640
target_height = 360

In [17]:
def process_frame(frame, confidence_threshold=0.5):
    resized_frame = cv2.resize(frame, (target_width, target_height))
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    inputs = feature_extractor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
    target_size = torch.tensor([image.size[::-1]]).to("cuda")
    results = feature_extractor.post_process_object_detection(outputs,
                                             target_sizes=target_size)[0]

    scores = results["scores"]
    labels = results["labels"]
    boxes = results["boxes"]

    keep= scores > confidence_threshold
    scores = scores[keep]
    labels = labels[keep]
    boxes = boxes[keep]

    keep_indices = nms(boxes, scores, iou_threshold)
    boxes = boxes[keep_indices]
    scores = scores[keep_indices]
    labels = labels[keep_indices]

    for score, label, box in zip(scores, labels, boxes):
        box = [round(i * (frame.shape[1] / target_width), 2) for i in box.tolist()]
        xmin, ymin, xmax, ymax = box

        cv2.rectangle(frame, (int(xmin)), (int(ymin)), (int(xmax)), (int(xmax)), color=(0, 255, 0), thickness=2)
        label_text = f"{model.config.id2label[label.item()]}: {round(score.item(), 2)}"
        cv2.putText(frame, label_text, (int(xmin), int(ymin - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame

if not cap.isOpened():
    print("Error opening video file")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("can't read frame")
        break
    processed_frame = process_frame(frame)
    out.write(processed_frame)
    cv2.imshow('Real-time Object Detection', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Error opening video file
can't read frame
